In [1]:
import os
import utils
import numpy as np
root = r'D:\Results\Zhong-et-al-2025'

exp_info = np.load(os.path.join(root, r'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()
exp_info.keys()

dict_keys(['unsup_train1_before_learning', 'unsup_train1_after_learning', 'sup_train1_before_learning', 'sup_train1_after_learning', 'unsup_test1', 'sup_test1', 'naive_test1', 'unsup_test2', 'sup_test2', 'naive_test2', 'unsup_test3', 'sup_test3', 'naive_test3', 'unsup_train2_before_learning', 'sup_train2_before_learning', 'unsup_train2_after_learning', 'sup_train2_after_learning', 'train1_before_grating', 'train1_after_grating', 'test1_before_grating', 'test1_after_grating', 'test2_before_grating', 'test2_after_grating'])

# Example mouse lick raster for before and after learning

In [ ]:
db = []
db.append(exp_inf.learning(mname='TX109')[0])
db.append(exp_inf.learning(mname='TX109')[-2])
db = np.array(db)


fn = 'example_bef_and_aft_learning_behavior.npy'
beh = get_behavior_object(db, is_imaging=0)
np.save(os.path.join(savePath, fn), beh)
print('Saving %s'%fn)

# Get selective index (dprime) for stimulus-selective neurons

In [2]:
exp_type = 'test1_after_grating' # type of experiment

stim_ID=[2, 3] # choose  
stim = ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3']

db = exp_info[exp_type] # get experiment information

# load behavior
Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()

# get dprime
all_dat = utils.Get_dprime_selective_neuron(db, Beh, stim_ID=stim_ID, root=root)

# save the dprime
fn = '%s_%s_%s_dprime.npy'%(exp_type, stim[stim_ID[0]], stim[stim_ID[1]])
np.save(os.path.join(root, 'process_data', fn), all_dat)
print(fn)

0 is finished
1 is finished
2 is finished
3 is finished
4 is finished
test1_after_grating_leaf1_leaf2_dprime.npy


# Get density map of selective neurons 

In [3]:
# exp_type = 'unsup_train2_FineDiscr_before' # type of experiment

# stim_ID=[3, 0] # choose stimuli: ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3'] 

typ = 'both' # 'both', 'stim1' or 'stim2'


stim = ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3']
fn = '%s_%s_%s_dprime.npy'%(exp_type, stim[stim_ID[0]], stim[stim_ID[1]])
# load saved dprime
dprimes = np.load(os.path.join(root, 'process_data', fn), allow_pickle=1).item()
dprime, retinotopy = dprimes['dprime'], dprimes['retinotopy']

# get density map
sel_map = utils.Get_density_map(dprime, retinotopy, dp_thr=0.3, typ=typ)
if typ=='stim1':
    saveN = fn.split('.')[0] + '_%s_distribution.npy'%(stim[stim_ID[0]])
elif typ=='stim2':
    saveN = fn.split('.')[0] + '_%s_distribution.npy'%(stim[stim_ID[1]])
else:
    saveN = fn.split('.')[0] + '_distribution.npy'
np.save(os.path.join(root, 'process_data', saveN), sel_map)
print(saveN)

test1_after_grating_leaf1_leaf2_dprime_distribution.npy


D:\Codes\zhong-et-al-2025\utils.py:321: RuntimeWarning: Mean of empty slice
  img = np.nanmean(np.array(img),axis=0) # mean across mice


# Get fraction of stimulus selective neurons above thresholds

In [4]:
# exp_type = 'unsup_train2_FineDiscr_before' # type of experiment

# stim_ID=[3, 0] # choose stimuli: ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3'] 


stim = ['circle1', 'circle2', 'leaf1', 'leaf2', 'leaf3']
fn = '%s_%s_%s_dprime.npy'%(exp_type, stim[stim_ID[0]], stim[stim_ID[1]])
# load saved dprime
dprimes = np.load(os.path.join(root, 'process_data', fn), allow_pickle=1).item()
dprime, retinotopy = dprimes['dprime'], dprimes['retinotopy']

frac = utils.Get_selective_neuron_fraction_with_dprime(dprime, retinotopy)

# save the fraction of selective neurons 
saveN = fn.split('.')[0] + '_frac.npy'
np.save(os.path.join(root, 'process_data', saveN), frac)
print(saveN)

test1_after_grating_leaf1_leaf2_dprime_frac.npy


# Get fraction and distribution of reward prediciton neurons above thresholds

In [3]:
exp_type = 'sup_train1_after_learning' # type of experiment

db = exp_info[exp_type] # get experiment information

root = r'D:\Results\Zhong-et-al-2025'
fn = '%s_leaf1_circle1_dprime.npy'%(exp_type)

# load behavior
Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()
# load saved stimulus selective dprime
dprimes = np.load(os.path.join(root, 'process_data', fn), allow_pickle=1).item()

frac, imgs = utils.Get_dprime_rewPred_neuron(db, Beh, dprimes, root=root, 
                              load_save_interp_spk=1, interp_spk_path=os.path.join(root, 'process_data'),
                             dp_thr=0.3)

# save the fraction of reward prediciton neurons 
save_fn0 = '%s_rew_frac.npy'%(exp_type)
np.save(os.path.join(root, 'process_data', save_fn0), frac)
save_fn1 = '%s_rew_distribution.npy'%(exp_type)
np.save(os.path.join(root, 'process_data', save_fn1), imgs)

{'sess#': 1, 'mname': 'VR2', 'datexp': '2021_04_06', 'blk': '1', 'is2p': 1, 'ROIdir': [], 'rewType': 'Passive', 'Note': '1-no cue in non-reward corridor', 'exptype': 'sup', 'artLick': 1, 'isDR': 0, 'stim_id': array([0, 2]), 'Gender': 'Male'}
load interpolated spk..
{'sess#': 1, 'mname': 'TX60', 'datexp': '2021_05_04', 'blk': '1', 'is2p': 1, 'ROIdir': [], 'rewType': 'Passive', 'exptype': 'sup', 'artLick': 1, 'isDR': 0, 'stim_id': array([0, 2]), 'Gender': 'Female'}
load interpolated spk..
{'sess#': 1, 'mname': 'TX108', 'datexp': '2023_03_22', 'blk': '1', 'is2p': 1, 'ROIdir': [], 'rewType': 'active after cue', 'exptype': 'sup', 'isDR': 0, 'stim_id': array([0, 2]), 'Gender': 'Male'}
load interpolated spk..
{'sess#': 1, 'mname': 'TX109', 'datexp': '2023_04_07', 'blk': '1', 'is2p': 1, 'ROIdir': [], 'rewType': 'active after cue', 'exptype': 'sup', 'isDR': 0, 'stim_id': array([2, 0]), 'Gender': 'Male'}
load interpolated spk..


# Save positional interpolation spks
### This will take a long time and space, but loading saved spks will save time for other data process

In [3]:
root = r'D:\Results\Zhong-et-al-2025'
exp_info = np.load(os.path.join(root, 'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()
exp_info.keys()
exp_type = 'unsup_train1_after_learning' # type of experiment

Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()

db = exp_info[exp_type] # get experiment information
n = 1
for ndb in db:
    if 'stimtype' in ndb.keys():
        kn = '%s_%s_%s_%s'%(ndb['mname'], ndb['datexp'], ndb['blk'], ndb['stimtype'])
    else:
        kn = '%s_%s_%s'%(ndb['mname'], ndb['datexp'], ndb['blk'])
    save_name = os.path.join(root, 'process_data', '%s_%s_%s_interpolate_spk.npy'%(ndb['mname'], ndb['datexp'], ndb['blk']))
    if os.path.exists(save_name):
        print("File has been created")
    else:    
        beh = Beh[kn]

        spk = utils.load_spk(ndb, root=os.path.join(root, 'spk'))
        nneu, nfr = spk.shape  

        ntrials, CL = beh['ntrials'], beh['Corridor_Length']    
        VRmove = beh['ft_move'][:nfr]>0
        ft_AcumPos = beh['ft_PosCum'][:nfr]

        utils.get_interpPos_spk(spk[:, VRmove], ft_AcumPos[VRmove], ntrials, n_bins=60, 
                                lengths=CL, save_path=save_name)
    print('done %d'%(n))
    n += 1

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\zhongl\AppData\Local\Temp\ipykernel_29840\4158800547.py:2: SyntaxWarning: invalid escape sequence '\I'
  exp_info = np.load(os.path.join(root, 'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()


File has been created
done 1
File has been created
done 2
10000
20000
30000
40000
50000
60000
done 3
File has been created
done 4
File has been created
done 5
File has been created
done 6
File has been created
done 7
File has been created
done 8
File has been created
done 9


# Get coding direction

In [2]:
root = r'D:\Results\Zhong-et-al-2025'
exp_info = np.load(os.path.join(root, 'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()
exp_info.keys()
exp_type = 'test2_after_grating' # type of experiment
db = exp_info[exp_type] # get experiment information

stim1, stim2 = 2, 3 # 'circle1':0, 'circle2':1, 'leaf1':2, 'leaf2':3

Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()
dat = utils.Get_coding_direction(db, Beh, stim_ref=[stim1, stim2], prc=5, root=root, 
                            load_save_interp_spk=1, interp_spk_path=os.path.join(root, 'process_data'), n_bef=10)

save_fn = '%s_coding_direction.npy'%(exp_type)
np.save(os.path.join(root, 'process_data', save_fn), dat)


<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\zhongl\AppData\Local\Temp\ipykernel_21032\3053096910.py:2: SyntaxWarning: invalid escape sequence '\I'
  exp_info = np.load(os.path.join(root, 'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()


load interpolated spk..
done normalization..
0
load interpolated spk..
done normalization..
1
load interpolated spk..
done normalization..
2
load interpolated spk..
done normalization..
3
load interpolated spk..
done normalization..
4


# Get sorted spikes with train and test for cross validation

In [3]:
# use 1/2 of trials (train) for getting dprime to define stimulus selective neurons;
# from the rest 1/2 trials, use half (tes1, 1/4 of total trials) to get and sort the peak positions
# sort the last 1/4 trials (test2) by the sorted index from test1
root = r'D:\Results\Zhong-et-al-2025'
exp_info = np.load(os.path.join(root, 'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()
exp_info.keys()

exp_type = 'naive_test3' # type of experiment
db = exp_info[exp_type] # get experiment information

stim1, stim2 = 2, 0 # 'circle1':0, 'circle2':1, 'leaf1':2, 'leaf2':3

Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()
dat = utils.Get_sort_spk(db, Beh, stim_ref=[2, 0], prc=5, root=root, 
                            load_save_interp_spk=1, interp_spk_path=os.path.join(root, 'process_data'))

save_fn = '%s_sort_spk.npy'%(exp_type)
np.save(os.path.join(root, 'process_data', save_fn), dat)

<>:5: SyntaxWarning: invalid escape sequence '\I'
<>:5: SyntaxWarning: invalid escape sequence '\I'
C:\Users\zhongl\AppData\Local\Temp\ipykernel_28564\3087677136.py:5: SyntaxWarning: invalid escape sequence '\I'
  exp_info = np.load(os.path.join(root, 'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()


load interpolated spk..
done normalization..
get dprime!


D:\Codes\zhong-et-al-2025\utils.py:597: RuntimeWarning: Mean of empty slice.
  targ_spk = sel_spk[:, tr_idx2].mean(1)
C:\Users\zhongl\AppData\Local\miniconda3\Lib\site-packages\numpy\_core\_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


0
load interpolated spk..
done normalization..
get dprime!
1
load interpolated spk..
done normalization..
get dprime!
2
load interpolated spk..
done normalization..
get dprime!
3
load interpolated spk..
done normalization..
get dprime!
4
load interpolated spk..
done normalization..
get dprime!
5
load interpolated spk..
done normalization..
get dprime!
6
load interpolated spk..
done normalization..
get dprime!
7
load interpolated spk..
done normalization..
get dprime!
8
load interpolated spk..
done normalization..
get dprime!
9


# Get stimulus selective neurons
## 1. Use 1/2 of trials (train) for getting dprime to define stimulus selective neurons;
## 2. Return the activity of other 1/2 of trials (test) 

In [3]:
root = r'D:\Results\Zhong-et-al-2025'
exp_info = np.load(os.path.join(root, 'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()
exp_info.keys()

exp_type = 'sup_train1_after_learning' # type of experiment
db = exp_info[exp_type] # get experiment information
ndb = db[2]
ndb

Beh = np.load(os.path.join(root, 'beh', 'Beh_'+ exp_type+ '.npy'), allow_pickle=1).item()
dat = utils.get_stimNeu_and_sorted(ndb, Beh['%s_%s_%s'%(ndb['mname'], ndb['datexp'], ndb['blk'])], 
                             stim_ref=[2, 0], thr=0.3, root=root, load_save_interp_spk=1)
save_fn = '%s_%s_%s_stimSelNeu_sorted.npy'%(ndb['mname'], ndb['datexp'], ndb['blk'])
np.save(os.path.join(root, 'process_data', save_fn), dat)

<>:2: SyntaxWarning: invalid escape sequence '\I'
<>:2: SyntaxWarning: invalid escape sequence '\I'
C:\Users\zhongl\AppData\Local\Temp\ipykernel_8340\3334289011.py:2: SyntaxWarning: invalid escape sequence '\I'
  exp_info = np.load(os.path.join(root, 'beh\Imaging_Exp_info.npy'), allow_pickle=1).item()


done loading spk..
load interpolation spk..
done loading interpolated spk..


# Get reward prediction neural activity; 10 folds cross validation

In [2]:
exp_type = 'sup_test3' # type of experiment

db = exp_info[exp_type] # get experiment information

Beh = utils.load_exp_beh(root, exp_type)
dat = utils.get_kfold_reward_response(root, db, Beh)

save_fn = '%s_reward_response.npy'%(exp_type)
np.save(os.path.join(root, 'process_data', save_fn), dat)

0 fold, neurons selected: 96
1 fold, neurons selected: 112
2 fold, neurons selected: 103
3 fold, neurons selected: 106
4 fold, neurons selected: 109
5 fold, neurons selected: 104
6 fold, neurons selected: 112
7 fold, neurons selected: 99
8 fold, neurons selected: 103
9 fold, neurons selected: 104
0 fold, neurons selected: 47
1 fold, neurons selected: 39
2 fold, neurons selected: 49
3 fold, neurons selected: 52
4 fold, neurons selected: 37
5 fold, neurons selected: 48
6 fold, neurons selected: 47
7 fold, neurons selected: 52
8 fold, neurons selected: 38
9 fold, neurons selected: 44
0 fold, neurons selected: 209
1 fold, neurons selected: 194
2 fold, neurons selected: 197
3 fold, neurons selected: 191
4 fold, neurons selected: 212
5 fold, neurons selected: 197
6 fold, neurons selected: 216
7 fold, neurons selected: 206
8 fold, neurons selected: 206
9 fold, neurons selected: 203
0 fold, neurons selected: 200
1 fold, neurons selected: 209
2 fold, neurons selected: 202
3 fold, neurons select

# Get reward prediction neural activity for single neuron

In [5]:
from importlib import reload
reload(utils)

<module 'utils' from 'D:\\Codes\\zhong-et-al-2025\\utils.py'>

In [6]:
exp_type = 'sup_test1' # type of experiment
db = exp_info[exp_type] # get experiment information
db = db[-1]
print(db)

Beh = utils.load_exp_beh(root, exp_type)
dat = utils.get_reward_neuorns(root, db, Beh['%s_%s_%s'%(db['mname'], db['datexp'], db['blk'])])

save_fn = 'Example_reward_neurons_in_%s.npy'%(exp_type)
np.save(os.path.join(root, 'process_data', save_fn), dat)

{'sess#': 1, 'mname': 'VR2', 'datexp': '2021_04_11', 'blk': '1', 'is2p': 1, 'ROIdir': [], 'rewType': 'Passive', 'artLick': 1, 'exptype': 'sup', 'isDR': 0, 'stim_id': array([0, 1, 2, 3]), 'Gender': 'Male'}
